In [1]:
import numpy as np
import datajoint as dj
import time
import pymeshfix
import os
import datetime
import calcification_Module as cm

#for supressing the output
import os, contextlib
import pathlib
import subprocess

#for error counting
from collections import Counter

#for reading in the new raw_skeleton files
import csv

from meshparty import trimesh_io

#for filtering
import math
from pykdtree.kdtree import KDTree

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')


Connecting celiib@10.28.0.34:3306


In [3]:
#function that takes in a 3x3 array of coordinates for faces and returns triangles and vertices
def index_unique_rows(full_coordinate_array):
    """
    Separates an array of nested coordinate rows into an array of unique rows and and index array.
    """
    vertices, flat_idx = np.unique(full_coordinate_array.reshape(-1, full_coordinate_array.shape[-1]), axis=0, return_inverse=True)
    return vertices, flat_idx.reshape(-1, full_coordinate_array.shape[-2])





In [14]:
"""Testing out null case """
# primary_key = dict(segmentation=2,segment_id=3456,compartment_type="Soma")
# vertices_soma,triangles_soma  = (ta3p100.CompartmentOrphan.ComponentOrphan() & primary_key).fetch("vertex_indices","triangle_indices")
# len(vertices_soma)

0

In [23]:
#will take in and populate the soma table based on the key it gets
def soma_verts_faces(query_key):
    
    table=""
    vertices_soma,triangles_soma = (ta3p100.CompartmentFinal.ComponentFinal() & query_key
                                    & "compartment_type='Soma'").fetch("vertex_indices","triangle_indices")

    if len(vertices_soma) > 0:
        print("Soma found in Exhitatory")
        #get the regular mesh from CleansedMesh
        vertices_mesh,triangles_mesh = (ta3p100.CleansedMesh & query_key).fetch("vertices","triangles")
    else:
        vertices_soma,triangles_soma = (ta3p100.CompartmentOrphan.ComponentOrphan() & query_key).fetch("vertex_indices","triangle_indices")
        if len(vertices_soma) > 0:
            print("Soma found in Orphans")
            vertices_mesh,triangles_mesh = (ta3p100.CleansedMeshOrphan & query_key).fetch("vertices","triangles")
        else:
            print("No Soma exists for " + str(query_key["segment_id"]))
            return np.array([]),np.array([])
            
    ts_flatten = np.hstack(triangles_soma)

    vertices_real = vertices_mesh[0]
    triangles_real = triangles_mesh[0]

    ts_stack_whole = vertices_real[triangles_real[ts_flatten]]

    vertices_whole, triangles_whole = index_unique_rows(ts_stack_whole)
    return vertices_whole, triangles_whole

In [49]:
def filter_edges_by_bounding_box(edges,max_bb_zone,min_bb_zone):
    #can just use bounding box function to get rid of any inside edges
    filtered_edges = list()

    for i,e in enumerate(edges):
        #print(e)
        if min(e[0][0],e[1][0])>max_bb_zone[0]:
            #print("minx>maxx")
            continue

        if max(e[0][0],e[1][0])<min_bb_zone[0]:
            #print("maxx<minx")
            continue

        if min(e[0][1],e[1][1])>max_bb_zone[1]:
            #print("miny>maxy")
            continue

        if max(e[0][1],e[1][1])<min_bb_zone[1]:
            #print("maxy<miny")
            continue

        if min(e[0][2],e[1][2])>max_bb_zone[2]:
            #print("minz>maxz")
            continue

        if max(e[0][2],e[1][2])<min_bb_zone[2]:
            #print("maxz<minz")
            continue

        filtered_edges.append(e)

    return np.array(filtered_edges)

In [53]:
ta3p100.FilteredSkeleton.describe()

-> ta3p100.Mesh
spine_filter_length  : int unsigned                 
---
n_edges              : int unsigned                 
edges                : longblob                     
lengths_of_removed_limbs : longblob                     



'-> ta3p100.Mesh\nspine_filter_length  : int unsigned                 \n---\nn_edges              : int unsigned                 \nedges                : longblob                     \nlengths_of_removed_limbs : longblob                     \n'

In [52]:
#find out what table 
ID = 648518346341393609
fake_ID = 6485183463413936090

primary_key = dict(segmentation=2,segment_id=ID)


start_time = time.time()
#create return key
return_key = primary_key.copy()

#pull down the skeleton for the mesh
skeleton_data = (ta3p100.FilteredSkeleton() & primary_key).fetch(as_dict=True)[0]
#skeleton_data = (ta3p100.FilteredSkeleton() & dict(segmentation=2,segment_id=ID)).fetch(as_dict=True)[0]

#get the vertices and triangles for the Soma
vertices_whole, triangles_whole = soma_verts_faces(primary_key)

#if no soma portion was found then just write regular skeleton
if not vertices_whole.any():
    
    #return_key["soma_exist"] = False
    return_key["vertices"] = vertices_whole
    return_key["triangles"] = triangles_whole
    return_key["edges"] = skeleton_data["edges"]
    return_key["n_edges"] = skeleton_data["n_edges"]
    return_key["soma_bounding_corners"] = np.array([])
    
    print(return_key)

#just need to strip the portions of the skeleton that are inside of the mesh

#find the bounding box
mesh = trimesh_io.Mesh(vertices=vertices_whole, faces=triangles_whole)
mesh.export("whole_mesh.off","off")
mesh.bounding_box.export("whole_mesh_bb.off","off")
min_bb = np.array(mesh.bounding_box.vertices).min(0)
max_bb = np.array(mesh.bounding_box.vertices).max(0)


#get the filtered edges according to bounding box:
filtered_edges_postsyn = filter_edges_by_bounding_box(skeleton_data["edges"],max_bb,min_bb)

print(f"Time until filtered edges = {time.time()-start_time}")

filtered_edges_postsyn.shape

#write off the new data to the table
#return_key["soma_exist"] = True
return_key["vertices"] = vertices_whole
return_key["triangles"] = triangles_whole
return_key["edges"] = filtered_edges_postsyn
return_key["n_edges"] = filtered_edges_postsyn.shape[0]
return_key["soma_bounding_corners"] = np.array((min_bb,max_bb))

print(filtered_edges_postsyn.shape)
filtered_edges_postsyn.shape[0]










Soma found in Exhitatory
Time until filtered edges = 0.8649575710296631
(56, 2, 3)


56

In [28]:
(ta3p100.FilteredSkeleton() & primary_key)

segmentation segmentation id,segment_id segment id unique within each Segmentation,spine_filter_length,n_edges,edges,lengths_of_removed_limbs
2,648518346341393609,2500,1326,=BLOB=,=BLOB=


In [27]:
#find out what table 
ID = 648518346341393609

primary_key = dict(segmentation=2,segment_id=ID)

#pull down the skeleton for the mesh
skeleton_data = (ta3p100.FilteredSkeleton() & primary_key).fetch(as_dict=True)
skeleton_data

[OrderedDict([('segmentation', 2),
              ('segment_id', 648518346341393609),
              ('spine_filter_length', 2500),
              ('n_edges', 1326),
              ('edges', array([[[390503. , 258004. ,  53092. ],
                       [390297. , 258083. ,  53343.4]],
               
                      [[360268. , 247349. ,  34906.8],
                       [362103. , 247974. ,  35768.2]],
               
                      [[371456. , 298452. ,  23197.6],
                       [371367. , 297628. ,  23062.2]],
               
                      ...,
               
                      [[372090. , 279077. ,  18538.1],
                       [371983. , 280015. ,  18723.2]],
               
                      [[372090. , 279077. ,  18538.1],
                       [372325. , 276610. ,  18883. ]],
               
                      [[373011. , 275048. ,  16765. ],
                       [372325. , 276610. ,  18883. ]]])),
              ('lengths_of_removed_l